# **ธนภัทร พูลโภคา 6710450911 หมู่ 200**

In [268]:
import pandas as pd
import numpy as np
import datetime

# 1. เปิดไฟล์ italy_earthquakes_from_2016-08-24_to_2016-11-30.csv

In [269]:
eq = pd.read_csv("italy_earthquakes_from_2016-08-24_to_2016-11-30.csv")
eq

,Time,Latitude,Longitude,Depth/Km,Magnitude
0,2016-08-24 03:36:32.000,42.6983,13.2335,8.1,6.0
1,2016-08-24 03:37:26.580,42.7123,13.2533,9.0,4.5
2,2016-08-24 03:40:46.590,42.7647,13.1723,9.7,3.8
3,2016-08-24 03:41:38.900,42.7803,13.1683,9.7,3.9
4,2016-08-24 03:42:07.170,42.7798,13.1575,9.7,3.6
...,...,...,...,...,...
8081,2016-11-30 18:39:27.600,42.8903,13.0197,10.7,2.0
8082,2016-11-30 18:43:14.850,42.9443,13.2003,8.6,2.6
8083,2016-11-30 20:18:27.550,43.0092,13.1288,8.0,2.2
8084,2016-11-30 20:45:11.780,43.0197,13.1017,9.3,2.7


# 2. ปรับคอลัมน์ Time ให้เป็น Timestamp และตั้งให้เป็น index

In [270]:
eq["Time"] = pd.to_datetime(eq["Time"])
eq.set_index("Time", inplace=True)
eq

,Latitude,Longitude,Depth/Km,Magnitude
Time,,,,
2016-08-24 03:36:32.000,42.6983,13.2335,8.1,6.0
2016-08-24 03:37:26.580,42.7123,13.2533,9.0,4.5
2016-08-24 03:40:46.590,42.7647,13.1723,9.7,3.8
2016-08-24 03:41:38.900,42.7803,13.1683,9.7,3.9
2016-08-24 03:42:07.170,42.7798,13.1575,9.7,3.6
...,...,...,...,...
2016-11-30 18:39:27.600,42.8903,13.0197,10.7,2.0
2016-11-30 18:43:14.850,42.9443,13.2003,8.6,2.6
2016-11-30 20:18:27.550,43.0092,13.1288,8.0,2.2


# 3. เวลาที่เกิดแผ่นดินไหวแรงที่สุดคือวันเวลาไหน

In [271]:
mx_datetime = eq.sort_values("Time", ascending=True).iloc[0, :].name
mx_datetime.strftime("%A %d %B %Y %H:%M:%S") + " - Magnitude " + str(eq.loc[mx_datetime, "Magnitude"])

'Wednesday 24 August 2016 03:36:32 - Magnitude 6.0'

# 4. ในวันที่เกิดแผ่นดินไหวแรงที่สุด มีแผ่นดินไหวกี่ครั้ง

In [272]:
eq[eq.index.date == mx_datetime.date()].sort_index()

,Latitude,Longitude,Depth/Km,Magnitude
Time,,,,
2016-08-24 03:36:32.000,42.6983,13.2335,8.1,6.0
2016-08-24 03:37:26.580,42.7123,13.2533,9.0,4.5
2016-08-24 03:40:46.590,42.7647,13.1723,9.7,3.8
2016-08-24 03:41:38.900,42.7803,13.1683,9.7,3.9
2016-08-24 03:42:07.170,42.7798,13.1575,9.7,3.6
...,...,...,...,...
2016-08-24 23:35:25.340,42.7577,13.0885,10.7,2.0
2016-08-24 23:39:06.170,42.6725,13.1917,9.6,2.0
2016-08-24 23:39:30.350,42.6053,13.2318,18.0,2.2


# 5. แสดงข้อมูลตั้งแต่ 7 ถึง 9 โมงเช้าในวันที่เกิดแผ่นดินไหวแรงที่สุด 

In [273]:
lower_mx_datetime = mx_datetime
upper_mx_datetime = mx_datetime
lower_mx_datetime = lower_mx_datetime.replace(hour=7).floor(freq="h")
upper_mx_datetime = upper_mx_datetime.replace(hour=9).floor(freq="h")
ranged_mx_date_eq = eq[(eq.index.date == mx_datetime.date()) & (eq.index.time >= lower_mx_datetime.time()) & (eq.index.time <= upper_mx_datetime.time())]

# 6. แผ่นดินไหวที่แรงที่สุดกับแรงเป็นที่ 2 ห่างกันเท่าไหร่ ตอบเป็นวัน-ชั่วโมง-นาที

In [274]:
diff = eq.sort_values(by="Magnitude").iloc[0].name - eq.sort_values(by="Magnitude").iloc[1].name
print("{:d}-{:02d}-{:02d}".format(diff.components.days, diff.components.hours, diff.components.minutes))

63-09-08


# 7. คำนวณความแรงของแผ่นดินไหวที่เปลี่ยนไปในแต่ละครั้ง ของข้อมูลในข้อ 5.
*หมายเหตุ: ใช้คำสั่ง shift() ช่วยเลื่อนแถวของคอลัมน์ Magnitude เพื่อคำนวณความแตกต่าง*

In [275]:
ranged_mx_date_eq = ranged_mx_date_eq["Magnitude"].shift(periods=[0,1], fill_value=0)
ranged_mx_date_eq["mag_diff"] = ranged_mx_date_eq["Magnitude_0"] - ranged_mx_date_eq["Magnitude_1"]
ranged_mx_date_eq["mag_diff"]

Time
2016-08-24 07:01:13.810    2.7
2016-08-24 07:02:24.570    0.7
2016-08-24 07:03:35.300   -1.1
2016-08-24 07:04:42.340    0.2
2016-08-24 07:05:28.100   -0.1
                          ... 
2016-08-24 08:51:30.830    0.4
2016-08-24 08:52:37.230   -0.1
2016-08-24 08:54:54.280    0.9
2016-08-24 08:59:37.370   -0.5
2016-08-24 08:59:45.360    0.1
Name: mag_diff, Length: 98, dtype: float64

# 8. มีแผ่นดินไหวเกิดขึ้นกี่ครั้งในแต่ละวัน

In [276]:
pd.Series(eq.index.date).value_counts()

2016-08-24    508
2016-10-31    461
2016-10-30    389
2016-10-27    370
2016-11-01    368
             ... 
2016-10-25      7
2016-10-23      7
2016-10-18      6
2016-10-12      6
2016-10-22      4
Name: count, Length: 99, dtype: int64

# 9. ถ้าแยกเวลาเป็นช่วง ช่วงละหนึ่งชั่วโมง มีกี่ช่วงที่ไม่มีแผ่นดินไหวเกิดขึ้นเลยในชั่วโมงนั้น

In [277]:
eq.resample("H").size().value_counts()[0]

C:\Users\thana\AppData\Local\Temp\ipykernel_24312\1929836011.py:1: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  eq.resample("H").size().value_counts()[0]


np.int64(759)

# 10. ถ้าแยกเวลาเป็นช่วง ช่วงละ 5 นาที ช่วงเวลาไหนมีค่าเฉลี่ยความรุนแรงของแผ่นดินไหวสูงที่สุด และค่าเฉลี่ยนั้นมีค่าเท่าไหร่

In [278]:
eq.resample("5min").mean()["Magnitude"].sort_values(ascending=False).head(1)

Time
2016-10-26 19:10:00    5.4
Name: Magnitude, dtype: float64

# 11. ระยะเวลาที่ไม่มีแผ่นดินไหวเลยนานที่สุดคือเท่าไหร่

In [279]:
eq["Time"] = eq.index
eq.sort_index(inplace=True)
eq["Time"].diff().sort_values(ascending=False).iloc[0]

Timedelta('0 days 16:46:16.250000')

# 12. ระยะเวลาที่ไม่มีแผ่นดินไหวเลยนานที่สุดคือช่วงไหน ระบุเป็นเวลาเร่ิมต้นและเวลาจบ

In [280]:
eq["TimeDiff"] = eq["Time"].diff()
mx_interval = eq["TimeDiff"].sort_values(ascending=False).iloc[0]
s_date = eq[eq.TimeDiff == mx_interval].index[0]
print(f"{s_date.strftime("%d %B %Y %H:%M:%S")} - {(s_date + mx_interval).strftime("%d %B %Y %H:%M:%S")}")

25 October 2016 20:00:10 - 26 October 2016 12:46:26


# 13. สมมติว่าเรานิยามแผ่นดินไหวที่รุนแรงนั้นต้องมี Magnitude มากกว่าการไหวครั้งก่อนและครั้งหลังอย่างน้อย 2.0 จงคัดเลือกเหตุการณ์แผ่นดินไหวที่รุนแรงออกมา

In [281]:
eq["MagDiff"] = eq["Magnitude"].diff()
eq_highdiff_mag = eq[eq.MagDiff >= 2.0]
eq_highdiff_mag

,Latitude,Longitude,Depth/Km,Magnitude,Time,TimeDiff,MagDiff
Time,,,,,,,
2016-08-24 04:33:28.890,42.7922,13.1507,8.0,5.4,2016-08-24 04:33:28.890,0 days 00:00:59.880000,2.9
2016-08-25 01:22:05.830,42.6535,13.2098,11.8,4.0,2016-08-25 01:22:05.830,0 days 00:14:08.610000,2.0
2016-10-26 19:10:36.340,42.8802,13.1275,8.7,5.4,2016-10-26 19:10:36.340,0 days 00:15:36.850000,3.3
2016-10-26 21:18:05.850,42.9087,13.1288,7.5,5.9,2016-10-26 21:18:05.850,0 days 00:04:42.570000,3.7
2016-10-27 10:21:45.600,42.8730,13.0965,9.4,4.3,2016-10-27 10:21:45.600,0 days 00:07:43.610000,2.0
2016-10-30 08:40:17.360,42.8322,13.1107,9.2,6.5,2016-10-30 08:40:17.360,0 days 00:08:57.470000,4.5
2016-11-01 09:56:39.600,42.9995,13.1583,9.9,4.8,2016-11-01 09:56:39.600,0 days 00:11:10.160000,2.8
2016-11-03 02:35:01.310,43.0285,13.0490,8.4,4.7,2016-11-03 02:35:01.310,0 days 00:06:55.180000,2.7
2016-11-29 18:14:02.690,42.5293,13.2803,11.1,4.4,2016-11-29 18:14:02.690,0 days 00:16:18.680000,2.0


# 14. จากตารางข้อ 13. ให้เพิ่มคอลัมน์ใหม่ชื่อ hour_count แสดงจำนวนแผ่นดินไหวในชั่วโมงที่มีเหตุการณ์แผ่นดินไหวรุนแรงด้วย

In [282]:
eq[eq.index.floor(freq="H").isin(eq_highdiff_mag.index.floor(freq="H"))]

C:\Users\thana\AppData\Local\Temp\ipykernel_24312\1470130597.py:1: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  eq[eq.index.floor(freq="H").isin(eq_highdiff_mag.index.floor(freq="H"))]
C:\Users\thana\AppData\Local\Temp\ipykernel_24312\1470130597.py:1: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  eq[eq.index.floor(freq="H").isin(eq_highdiff_mag.index.floor(freq="H"))]


,Latitude,Longitude,Depth/Km,Magnitude,Time,TimeDiff,MagDiff
Time,,,,,,,
2016-08-24 04:00:00.450,42.6125,13.2497,11.2,2.9,2016-08-24 04:00:00.450,0 days 00:00:36.820000,0.1
2016-08-24 04:01:08.120,42.7988,13.2412,6.9,3.9,2016-08-24 04:01:08.120,0 days 00:01:07.670000,1.0
2016-08-24 04:03:56.210,42.6253,13.2545,12.7,2.6,2016-08-24 04:03:56.210,0 days 00:02:48.090000,-1.3
2016-08-24 04:04:05.850,42.7928,13.1333,9.8,2.8,2016-08-24 04:04:05.850,0 days 00:00:09.640000,0.2
2016-08-24 04:04:18.860,42.7892,13.1233,10.1,3.1,2016-08-24 04:04:18.860,0 days 00:00:13.010000,0.3
...,...,...,...,...,...,...,...
2016-11-29 18:29:27.570,42.5435,13.2753,12.5,2.0,2016-11-29 18:29:27.570,0 days 00:06:24.650000,-0.1
2016-11-29 18:31:33.060,42.8103,13.0797,8.1,2.3,2016-11-29 18:31:33.060,0 days 00:02:05.490000,0.3
2016-11-29 18:35:53.980,42.7307,13.2412,11.9,2.2,2016-11-29 18:35:53.980,0 days 00:04:20.920000,-0.1


# 15. แสดงจำนวนแผ่นดินไหวแยกตาม 24 ชั่วโมง (เอาจำนวนแผ่นดินไหวของแต่ละชั่วโมง 0-23 ของทุกวันรวมกัน แสดงผลตามชั่วโมง)

In [283]:
eq.groupby(by=eq.index.hour)["Magnitude"].sum()

Time
0     775.7
1     714.2
2     719.3
3     743.4
4     857.1
5     958.6
6     969.0
7     943.9
8     886.5
9     962.6
10    818.2
11    784.5
12    806.6
13    764.5
14    842.8
15    775.6
16    712.6
17    754.3
18    743.2
19    813.6
20    741.3
21    758.0
22    753.9
23    793.8
Name: Magnitude, dtype: float64

# 16. คำนวณค่าเฉลี่ยของความรุนแรงแผ่นดินไหวที่มากที่สุดในช่วง 3 วัน โดยหาความแรงของแผ่นดินไหวที่มากที่สุดในแต่ละวันก่อน แล้วใช้ sliding window หาค่าเฉลี่ยของทุก ๆ 3 วัน

In [284]:
eq.rolling("3D")["Magnitude"].mean().sort_values(ascending=False)

Time
2016-08-24 03:36:32.000    6.000000
2016-08-24 03:37:26.580    5.250000
2016-08-24 03:40:46.590    4.766667
2016-08-24 03:41:38.900    4.550000
2016-08-24 03:42:07.170    4.360000
                             ...   
2016-09-28 10:48:24.070    2.193333
2016-09-29 08:29:17.490    2.192857
2016-09-29 07:49:19.070    2.192857
2016-09-29 09:51:58.680    2.186667
2016-09-29 08:48:15.710    2.186207
Name: Magnitude, Length: 8086, dtype: float64